# Comparação de resultados: LLM x Z3
Este notebook consolida os registros salvos pelo pipeline e apresenta uma visão rápida de como cada modelo se comporta ao resolver os puzzles Knights and Knaves em comparação com a solução obtida via Z3.

## Como usar
1. Gere puzzles e execute `main.py`, `main_gemini.py` ou `main_gpt.py` para popular os arquivos `resultados/*.jsonl`.
2. Reabra ou reexecute este notebook para atualizar as métricas e gráficos.

In [ ]:
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-muted')
pd.options.display.max_colwidth = 120


In [ ]:
def load_jsonl(path: str, model_label: str) -> pd.DataFrame:
    file_path = Path(path)
    if not file_path.exists():
        print(f"Aviso: arquivo {path} não encontrado.")
        return pd.DataFrame()

    rows = []
    with file_path.open(encoding='utf-8') as fp:
        for line in fp:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))

    if not rows:
        print(f"Aviso: arquivo {path} está vazio.")
        return pd.DataFrame()

    df = pd.DataFrame(rows)
    if 'model' in df.columns:
        df['model_name'] = df['model']
    else:
        df['model_name'] = model_label

    df['match'] = df['match'].astype(bool)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df


In [ ]:
gemini_df = load_jsonl('resultados/results.jsonl', 'gemini-2.5-flash')
gpt_df = load_jsonl('resultados/results_gpt.jsonl', 'gpt-4o-mini')

df = pd.concat([gemini_df, gpt_df], ignore_index=True)
df = df.sort_values('timestamp') if not df.empty else df
df.head()


## Visão geral por modelo

In [ ]:
if df.empty:
    print('Sem dados disponíveis. Execute os scripts principais antes de rodar este notebook.')
else:
    resumo = (
        df.groupby('model_name')['match']
        .agg(total='count', acertos='sum')
        .assign(taxa_acerto=lambda x: (x['acertos'] / x['total'] * 100).round(1))
        .sort_values('taxa_acerto', ascending=False)
    )
    display(resumo)


In [ ]:
if not df.empty:
    fig, ax = plt.subplots(figsize=(6, 4))
    resumo.reset_index().plot(
        x='model_name', y='taxa_acerto', kind='bar', color='#4c72b0', legend=False, ax=ax
    )
    ax.set_ylabel('Taxa de acerto (%)')
    ax.set_xlabel('Modelo')
    ax.set_ylim(0, 100)
    ax.set_title('Precisão por modelo')
    for idx, value in enumerate(resumo['taxa_acerto']):
        ax.text(idx, value + 1, f"{value:.1f}%", ha='center')
    plt.tight_layout()
    plt.show()


## Desempenho por puzzle

In [ ]:
if not df.empty:
    puzzle_summary = (
        df.groupby(['puzzle_file', 'model_name'])['match']
        .agg(total='count', acertos='sum')
        .assign(taxa_acerto=lambda x: (x['acertos'] / x['total'] * 100).round(1))
        .reset_index()
        .sort_values(['puzzle_file', 'model_name'])
    )
    display(puzzle_summary)


## Onde os modelos erraram

In [ ]:
if not df.empty:
    erros = df[~df['match']]
    if erros.empty:
        print('Nenhum erro registrado até o momento.')
    else:
        display(
            erros[
                ['timestamp', 'model_name', 'puzzle_file', 'llm_answer', 'z3_answer']
            ].sort_values('timestamp')
        )


Pronto! Sempre que gerar novos resultados, reexecute todas as células para atualizar as tabelas e gráficos.